## Counter

To implement a counter we can use the `Register` primitive and the `+` operator. How can we make a more general counter of arbitray width?

In [1]:
import magma as m
from mantle import Register

/Users/hanrahan/miniconda3/lib/python3.7/site-packages/pysmt/walkers/generic.py:43: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if len(nodetypes) == 1 and isinstance(nodetypes[0], collections.Iterable):


The generator for a counter is parameterized by the number of bits in the counter, `n`.

In [2]:
def DefineCounter(n):
    class _Counter(m.Circuit):
        name = f'Counter{n}'
        IO = ["O", m.Out(m.UInt[n])] + m.ClockInterface()
        @classmethod
        def definition(io):
            reg = Register(n, _type=m.UInt)
            io.O <= reg(reg.O + 1)
            m.wireclock(io, reg)
            m.wiredefaultclock(io, reg)
    return _Counter

Sinulate the counter.

In [3]:
from magma.simulator import PythonSimulator

Counter4 = DefineCounter(4)
sim = PythonSimulator(Counter4, Counter4.CLK)

sim.evaluate()
val = sim.get_value(Counter4.O)
print(sim.get_value(Counter4.O))
assert val == 0

for i in range(1,11): 
    sim.advance() # toggle clock - now High
    assert i == sim.get_value(Counter4.O)
    print(sim.get_value(Counter4.O))

    sim.advance() # toggle clock - now Low
    assert i == sim.get_value(Counter4.O)

print("Success!")

ValueError: Cannot simulate magma_Bits_4_add_inst0<magma_Bits_4_add(I0: In(UInt[4]), I1: In(UInt[4]), O: Out(UInt[4]))> of type magma_Bits_4_add(I0: In(UInt[4]), I1: In(UInt[4]), O: Out(UInt[4])) because it does not have a Python simulate method

In [ ]:
m.compile("build/Counter4", Counter4, output="coreir")
!coreir -i build/Counter4.json -p instancecount